In [1]:
training_data = [
    ['Green',3,'Apple'],
    ['Yellow',3,'Apple'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon']
]

In [2]:
header = ['Color','Diameter',"Label"]

In [3]:
def unique_vals(rows,col):
    return set([row[col] for row in rows])

In [4]:
unique_vals(training_data,0)

{'Green', 'Red', 'Yellow'}

In [5]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [6]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [7]:
def is_numeric(value):
    return isinstance(value,int) or isinstance(value,float)

In [8]:
class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
    
    def match(self,example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
        header[self.column],condition,str(self.value))

In [9]:
Question(0,"Green")

Is Color == Green?

In [10]:
q = Question(0, 'Green')

In [11]:
q

Is Color == Green?

In [12]:
example = training_data[0]

In [13]:
q.match(example)

True

In [14]:
def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [15]:
true_rows,false_rows = partition(training_data,Question(0,'Red'))

In [16]:
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [17]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [18]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [19]:
pure = [['Apple'],['Apple']]

In [20]:
dict1 = class_counts(pure)

In [21]:
for lbl in dict1:
    print('hi')

hi


In [22]:
len(pure)

2

In [23]:
gini(pure)

0.0

In [24]:
mixing = [['Apple'],['Grape'],['']]

In [25]:
gini(mixing)

0.6666666666666665

In [26]:
def info_gain(left,right,current_uncertainity):
    p = float(len(left)/ len(left) + len(right))
    return current_uncertainity- p *gini(left) - (1-p)*gini(right)

In [27]:
current_uncertainity = gini(training_data)

In [28]:
current_uncertainity

0.6399999999999999

In [29]:
true_rows, false_rows = partition(training_data,Question(0,'Green'))

In [30]:
true_rows

[['Green', 3, 'Apple']]

In [31]:
false_rows

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [32]:
info_gain(true_rows,false_rows,current_uncertainity)

3.1399999999999997

In [35]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows,false_rows,current_uncertainity)
            if gain >= best_gain:
                best_gain,best_question = gain,question
                
    return best_gain,best_question
            
    

In [36]:
best_gain,best_question = find_best_split(training_data)

In [37]:
best_gain

3.1399999999999997

In [38]:
best_question

Is Color == Green?

In [39]:
class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [40]:
class Decision_node:
    
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [41]:
def build_trees(rows):
    
    gain,question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    
    true_rows,false_rows = partition(rows,question)
    true_branch  = build_trees(true_rows)
    false_branch = build_trees(false_rows)
    
    return Decision_node(question,true_branch,false_branch)
    

In [42]:
def print_tree(node,spacing=" "):
    
    if isinstance(node,Leaf):
        print(spacing + "Predict",node.predictions)
        return
    print(spacing + str(node.question))
    
    print(spacing + '==> True:')
    print_tree(node.true_branch,spacing + "  ")
    
    print(spacing + '==> False:')
    print_tree(node.false_branch,spacing + "  ")
    

In [43]:
my_tree = build_trees(training_data)

In [44]:
print_tree(my_tree)

 Is Color == Green?
 ==> True:
   Predict {'Apple': 1}
 ==> False:
   Is Color == Red?
   ==> True:
     Predict {'Grape': 2}
   ==> False:
     Predict {'Apple': 1, 'Lemon': 1}


In [45]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [46]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [47]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Apple': '100%'}

In [48]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{'Apple': '50%', 'Lemon': '50%'}

In [49]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [50]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
